### 5.1.3 Stratified K-Fold cross-validation and other strategies
- 계층별 K-Fold 교차 검증
  - 각 Fold안의 클래스 비율이 전체 원본 데이터셋에 있는 클래스 비율과 동일하도록 맞춤
  - 즉, 원본 데이터셋에서 클래스 A가 90%, 클래스 B가 10% 비율이라면, 계층별 K-Fold 교차 검증에서 각 K개의 Fold안에는 클래스 A가 90%, 클래스 B가 10% 비율이 됨.
- scikit-learn의 cross_val_score 기본 설정
  - 분류모델: StratifiedKFold를 사용하여 기본적으로 계층별 K-Fold 교차 검증 수행
  - 회귀모델: 단순한 KFold를 사용하여 계층별이 아닌 기본 K-Fold 교차 검증 수행
    - 대신 회귀모델에서는 KFold를 사용할 때 shuffle 매개변수를 True로 지정하여 폴드를 나누기 전에 무작위로 데이터를 섞는 작업 추천

In [1]:
%matplotlib inline
import sys 
sys.path.append('..')

from preamble import *
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()
print("Iris labels:\n{}".format(iris.target))

Iris labels:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


### More control over cross-validation
- 기본적으로...
  - 분류: StratifiedKFold가 사용됨
  - 회귀: KFold가 사용됨
- 하지만, 때때로 분류에 KFold가 사용되어야 할 필요도 있음
  - 다른 사람이 이미 수행한 사항을 재현해야 할 때
  - StratifiedKFold가 아닌 KFold를 생성하여 cross_val_score()의 cv 인자에 할당

In [3]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5) #교차 검증 분할기의 역할 수행

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

scores = cross_val_score(logreg, iris.data, iris.target)
print("Cross-validation scores:\n{}".format(cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

Cross-validation scores:
[1.    0.933 0.433 0.967 0.433]


- 이런 경우 3-Fold를 사용하면 데이터 타겟 레이블 분포 특성상 성능이 매우 나쁠 수 있음

In [5]:
kfold = KFold(n_splits=3)
print("Cross-validation scores:\n{}".format(cross_val_score(logreg, iris.data, iris.target, cv=kfold)))


Cross-validation scores:
[0. 0. 0.]


- 해결책
  - KFold를 만들 때 shuffle=True를 통해 데이터를 임의로 섞음.
  - random_state=0을 주면 추후 그대로 재현이 가능

In [6]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("Cross-validation scores:\n{}".format(cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

Cross-validation scores:
[0.9  0.96 0.96]


#### Leave-one-out cross-validation (LOOCV)
- Fold 하나에 하나의 샘플이 들어 있는 Stratified k-Fold 교차 검증
  - 즉, 각각의 반복에서 테스트 데이터에 하나의 샘플만 존재
  - 데이터셋이 클 때 시간이 매우 오래 걸림
  - 작은 데이터셋에 대해서는 일반적인 상황에 대한 거의 확실한 score 값을 얻을 수 있음.

In [7]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(logreg, iris.data, iris.target, cv=loo)
print("Number of cv iterations: ", len(scores))
print("Mean accuracy: {:.2f}".format(scores.mean()))

Number of cv iterations:  150
Mean accuracy: 0.95


#### Shuffle-split cross-validation
- 임의 분할 교차 검증
  - model_selection.SuffleSplit(n_splits=10, test_size='default') or model_selection.StratifiedSuffleSplit(n_splits=10, test_size='default')
    - n_splits: 10
      - 분할의 개수
    - test_size 만큼의 테스트 셋트를 만들도록 분할
      - test_size의 기본값: 0.1
  - 보통 test_size 값만 설정하며, 추가적으로 train_size 도 설정 가능
    - 이런 경우 전체 데이터 집합 중 일부만 훈련과 테스트에 사용할 수 있음
    - 대규모 데이터에 유용
  - test_size, train_size
    - 정수: 데이터 포인트의 개수
    - 실수: 데이터 포인트 비율<br/><br/>

- 아래 그림 예제
  - 전체 데이터 셈플 개수: 10
  - train_size = 5
  - test_size = 2
  - n_splits = 4

In [8]:
from sklearn.model_selection import ShuffleSplit

shuffle_split = ShuffleSplit(n_splits=10, test_size=.5, train_size=.5)
scores = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)

print("Cross-validation scores:\n{}".format(scores))
print("Mean accuracy: {:.2f}".format(scores.mean()))

Cross-validation scores:
[0.84  0.933 0.947 0.96  0.893 0.973 0.947 0.933 0.973 0.947]
Mean accuracy: 0.93


In [9]:
from sklearn.model_selection import StratifiedShuffleSplit

shuffle_split = StratifiedShuffleSplit(n_splits=10, test_size=.5, train_size=.5)
scores = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)

print("Cross-validation scores:\n{}".format(scores))
print("Mean accuracy: {:.2f}".format(scores.mean()))

Cross-validation scores:
[0.947 0.973 0.933 0.947 0.947 0.96  0.947 0.92  0.947 0.907]
Mean accuracy: 0.94
